# Documentation On Identifying Fraud from Enron Email

## Yong Yu
---

## Overview

Overall, the building process in this project can be broke into two parts. The first part is choosing best combination of feature scaling methods, feature selection methods, and classification method. In particular, this part includes checking on different feature selection methods with different but all well tuned classifers, excluding or including PCA, before scaling features using various ways. The second part is to further tuning feature selection paramters, PCA parameters to increase the performance of the estimator.

The final model uses a pipeline made of MinMaxScaler, PCA, ExtraTreeClassifier, and LinearSVC, with cross validation method StratifiedShuffleSplit. The details are as following,

<pre>[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
 ('extra_tree_with_pca', FeatureUnion(n_jobs=1, transformer_list=[
 ('pca', PCA(copy=True, n_components=1, whiten=False)),
 ('extra_tree', ExtraTreesClassifier(bootstrap=False, class_weight='auto', criterion='gini',
                                     max_depth=None, max_features=2, max_leaf_nodes=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=1,
                                     oob_score=False, random_state=42, verbose=0, warm_start=False))],
                                     transformer_weights=None)),
 ('linear_svc', LinearSVC(C=0.1, class_weight='auto', dual=False, fit_intercept=True,
                          intercept_scaling=1, loss='squared_hinge', max_iter=1000.0,
                          multi_class='ovr', penalty='l1', random_state=42, tol=0.0001,verbose=0))]</pre>

## About This Project
The goal of this project is to identify whether a person is guilty for the notorious Enron Fraud, using publicly available information like finacial incomes and emails. 

From oil-trading, bandwidth trading, and weather trading with market-to-market accounting, Enron has moved its hands to a variety of commodities, got in touch with politicians like George W. Bush, and caused a great loss to the public, including the California electricity shortage. All these information can be useful if text learning was applied, and certain patterns could be found out like, a pattern indicating a decision-making person could very likely be a person of interest. However, this is not applied in this analysis since it's a more advanced topic.

This analysis used a finacial dataset containing people's salary, stock information, and so on. During Enron Fraud, people like Jefferey Skilling, Key Lay, and Fastow all have dumped large amounts of stock options, and they are all guilty. This information can be very helpful to check on other person of interest, and can be easily refelected in the dataset. This is also where machine learning comes into play. By creating models which calculate relationships between a person of interest and its available quantitative data, machine learning tries to find and memorize a pattern that helps us identify a guilty person in the future.

##About This Dataset

The following is a list of findings,
- there are 146 data points with 21 features, and a total of 3066 obervations.
- there are 18 people who is an point of interest.
- 1,358 data points are missing.
- the top 3 features with most missing values are "loan_advances", "director_fees", and "restricted_stock_deferred".

There are certain outliers by each feature. One big outlier was found to be "TOTAL" and certainly it was removed. To find out other outliers, a multivariable linear regression model was created. By removing data points with top 10% variance between predicted and real values, an outlier-cleaned dataset could be created for the analysis. However, as there were 13 person-of-interest out of total 18 appearing in the outliers, simply removing the outliers before spliting it into training and testing set would weaken the model as it won't have much work to do when identifying POIs. Given this thought, the outlier cleaning was only performed on the training set, and the cleaned amount is down to 5%.

## Feature Scaling

Three different feature scaling methods are explored in this report, including MinMaxScaler, StandardScaler, and Normalizer. The MinMaxScaler is chosen for the final feature scaling method.

Generally speaking, it is crucial to scale the features when a distance function is used as training algorithm while it's not necessary for a linear function, 
<a href='http://www.faqs.org/faqs/ai-faq/neural-nets/part2/section-16.html' target='_blank'>although scaling would make it faster.</a>
If a feature has a scale of 0 to 100, and a second feature has a scale of 0 to 100,000, the contribution from the first feature would then be totally swamped by the second one. Thus it's essential to scale them to a same level before training on a distance based algorithm. In this dataset, LinearSVC is chosen in the final model, which is a distance based algorithm. And the standard deviation among features could be as high as 8846594(total_payments), and as low as 74(from_poi_to_this_person), therefore it's necessary to scale the dataset.

## Feature Selection and PCA

The feature selection method for the final model is ExtraTreeClassifier. After tuning, this model takes n_estimators = 1 and max_features = 2 as optimal parameter values, and 11 features are chosen, their feature importances are listed below,

- restricted_stock: 0.3797
- expenses: 0.1766
- shared_receipt_with_poi: 0.1462
- bonus: 0.1040
- from_poi_to_this_person: 0.0444
- total_stock_value: 0.0419
- other: 0.0419
- from_this_person_to_poi: 0.0234
- exercised_stock_options: 0.0200
- deferral_payments: 0.0162
- loan_advances: 0.0052

For PCA, after tuning, it chose n_components = 1, and the explained variance ratio is 0.3568.

## Feature Engineering

Three new features, "stock_salary_ratio", "poi_from_ratio", "poi_to_ratio", are created when analyzing the data.

stock_salary_ratio takes the result from total_stock_value divided by salary. This feature is useful based on the assumption that a person of interest usually has a unusual large stock value since it's under the table, while salary information could be more easily known by public, thus the ratio could give information to identify the poi. The bigger the ratio, the more likely it is a poi.

poi_from_ratio takes result from from_poi_to_this_person divided by from_messages. This feature assumes that if a person is a poi, he/she tends to have more contacts with another poi, therefore the ratio would be bigger. And same applie to feature poi_to_ratio.

Before any final tuning on feature scaling methods, their feature importances are as following,

- poi_from_ratio: 0.0420
- poi_to_ratio: 0.0292
- stock_salary_ratio: 0.0185

And the model performance after adding these new features,

$$Accuracy: 0.7697, Precision: 0.2495, Recall: 0.362, F1: 0.2954, F2: 0.3321$$

whose precision and recall scores are significant lower than training with these new features,

$$Accuracy: 0.7643, Precision: 0.3042, Recall: 0.5965, F1: 0.4029, F2: 0.5003$$

Therefore, these features are not used in the final model.

## Classifiers

The chosen classifer is LinearSVC, after scaled by MinMaxScaler, and using PCA and ExtraTreeClassifier. LinearSVC is compared with KNeighborsClassifer. From the tuning results, KNeighborClassifier tends to give a higher precision score, while LinearSVC gives a higher recall score. And KNeighborClassifer always gives a higher accuracy score. However, since LinearSVC tends to give a higher F1 score, based on the list below, LinearSVC was chosen as it gave the highest F1 score after combining with feature scaling and feature selection.

In [4]:
import pandas as pd
pd.read_csv("model_metrix.csv").sort(['f1_score', 'time_used'], ascending=[0,1])

,model,scaled,feature_selection_method,classification_method,accuracy_score,f1_score,precision_score,recall_score,time_used
7,23,minmaxscaler,extra_tree_with_pca,linear_svc,0.7643,0.4029,0.3042,0.5965,106.815
2,21,minmaxscaler,k_best_with_pca,linear_svc,0.7845,0.3010,0.2652,0.3480,37.172
8,15,standardscaler,extra_tree_with_pca,linear_svc,0.7456,0.2891,0.2304,0.3880,115.377
3,4,none,extra_tree,k_neighbors,0.8556,0.2833,0.4188,0.2140,39.891
5,19,minmaxscaler,extra_tree,linear_svc,0.8045,0.2641,0.2651,0.2630,97.704
6,11,standardscaler,extra_tree,linear_svc,0.7473,0.2623,0.2147,0.3370,100.835
9,7,none,extra_tree_with_pca,linear_svc,0.8041,0.2470,0.2533,0.2410,234.325
1,9,standardscaler,k_best,linear_svc,0.7870,0.2281,0.2207,0.2360,33.350
4,5,none,k_best_with_pca,linear_svc,0.7863,0.2268,0.2191,0.2350,81.568
0,17,minmaxscaler,k_best,linear_svc,0.7785,0.2216,0.2085,0.2365,32.203


## Tuning

Tuning the algorithm means to find a better solution specificly for the current problem. If an algorithm is not well tuned, it could lower down the accuracy score, percision and recall scores, which means a better outcome would be missed. Even worse, it could increase the runtime.

For the final model, three algorithms are carefully tuned. For LinearSVC, the parameters tuned are 'C', 'tol', and 'max_iter'. For ExtraTreeClassifier, the parameters tuned are 'n_estimators' and 'max_features'. For PCA, the parameter n_components is tuned.

When tuning LinearSVC, C was dominating the performance alone. The algorithm was found to be best at C = 0.1. At C = 1.0, tol started to make positive effect. At C = 3.0, model performance decreased when tol changed from 0.1 to 1.0. At C = 10.0, model performance decreased when tol changed from 0.01 to 0.1, and increased when it got to 1.0. For all the fitting, max_iter didn't take any effect on performance.

When tuning ExtraTreeClassifer, the performance generally went up when n_estimators increased, and had a complicated relationship with max_features. However, at max_features = 2 and n_estimators = 1, a best performance was found within the given range.

When tuning PCA, the n_components had no effect on model performance when whiten = False. When whiten = True, the performance stayed the same when n_components was less than 6, and started to decrease while n_components was greater than 5 and increased. The best parameters were found to be n_components = 1 and whiten = True.

## Validation
Validation is to hold a part of a dataset, fit the model on the rest of the data, and predict using the held data. The classic mistake, overfitting, would happen if no cross validation is applied.

In this model, StratifiedShuffleSplit is used as the cross validation method. It is chosen as the data is heavily imbalanced, as there are only 18 POIs and 125 non-POIs. Stratifying it makes sure when splitting the dataset, the classes in training and testing sets are proportional.

Considering time consumption, when tuning algorithms and evaluating models, n_iter = 100 and n_iter = 1000 were used respectively.

## Evaluation
The evaluation metrics in this analysis used are accuaracy score, f1 score, precision score and recall score.

The final model has an accuracy score of 0.7404, which means 74.04% of the predictions are found to be true. However, using this metrix won't give us very valuable information. The dataset is heavily imbalance and guessing all outcomes to be non-POIs would give an accuracy of 86.2% already, this is why we need use precision and recall scores instead.

With a precision score of 0.31, it tells us that if this model predicts 100 POIs, then the chance would be 31 people who are truely POIs and the rest 69 are innocent. On the other hand, with a recall score of 0.76, this model can find 76% of all real POIs in prediction. This model is good at finding bad guys, with the price of high probability(0.69) in calumniating someone.

Finally, there's always a tradeoff between precision and recall, f1 comes and measure how well the tradeoff is. With a f1 score of 0.44, this model is ok, while more improvements could be applied such as better feature engineering.